## Word count with hdfs3, distributed, and dask

In this example, we count the number of words in text files (Enron email dataset - 6.4 GB) stored in HDFS.

We use the libraries in an increasing order of API functionality, from low-level to high-level:

* hdfs3
* hdfs3 + distributed
* hdfs3 + distributed + dask

Setup:

```
conda install hdfs3 -c blaze
pip install dask distributed
```

### Example 1) Word count with hdfs3

In [1]:
import hdfs3

In [2]:
hdfs = hdfs3.HDFileSystem('ip-172-31-56-96', port=8020)

Generate list of foldernames and filenames in /tmp/enron

In [3]:
dirnames = [x['name'] for x in hdfs.ls('/tmp/enron')]

In [4]:
dirnames[:5]

[b'/tmp/enron/edrm-enron-v2_allen-p_xml.zip',
 b'/tmp/enron/edrm-enron-v2_arnold-j_xml.zip',
 b'/tmp/enron/edrm-enron-v2_arora-h_xml.zip',
 b'/tmp/enron/edrm-enron-v2_badeer-r_xml.zip',
 b'/tmp/enron/edrm-enron-v2_bailey-s_xml.zip']

In [5]:
filenames = [x.decode('utf-8') + '/merged.txt' for x in dirnames]

In [6]:
filenames[:5]

['/tmp/enron/edrm-enron-v2_allen-p_xml.zip/merged.txt',
 '/tmp/enron/edrm-enron-v2_arnold-j_xml.zip/merged.txt',
 '/tmp/enron/edrm-enron-v2_arora-h_xml.zip/merged.txt',
 '/tmp/enron/edrm-enron-v2_badeer-r_xml.zip/merged.txt',
 '/tmp/enron/edrm-enron-v2_bailey-s_xml.zip/merged.txt']

Print first 10 lines of first file

In [7]:
with hdfs.open(filenames[0]) as f:
    f.encoding = 'utf-8'
    [print(f.readline()) for i in range(10)]

Date: Tue, 26 Sep 2000 09:26:00 -0700 (PDT)
From: Phillip K Allen
To: pallen70@hotmail.com
Subject: Investment Structure
X-SDOC: 948896
X-ZLID: zl-edrm-enron-v2-allen-p-1713.eml

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 
04:26 PM ---------------------------



In [8]:
def count_words(filename, encoding='utf-8'):
    with hdfs.open(filename) as f:
        f.encoding = encoding
        count = 0
        all_lines = f.readlines()
        for line in all_lines:
            words = line.split()
            count += len(words)
    return count

Count words in first file

In [9]:
%%time
count_words('/tmp/enron/edrm-enron-v2_allen-p_xml.zip/merged.txt')

CPU times: user 3.25 s, sys: 151 ms, total: 3.4 s
Wall time: 3.41 s


13099980

Count words in all (readable) files

In [10]:
%%time
wordcounts = {}
for filename in filenames:
    try:
        wordcounts[filename] = count_words(filename)
    except UnicodeDecodeError:
        wordcounts[filename] = 'Encoding error'

CPU times: user 1min 52s, sys: 7.95 s, total: 1min 59s
Wall time: 2min


In [11]:
wordcounts.values()

dict_values([640160, 712714, 602964, 833143, 435731, 4179444, 7071894, 2010407, 'Encoding error', 'Encoding error', 0, 290147, 1502415, 10528904, 5736422, 'Encoding error', 'Encoding error', 'Encoding error', 0, 'Encoding error', 'Encoding error', 1778714, 8222368, 'Encoding error', 5803416, 'Encoding error', 'Encoding error', 'Encoding error', 3463107, 416404, 6019540, 5814482, 'Encoding error', 4746877, 551252, 1082150, 'Encoding error', 10179741, 'Encoding error', 1302679, 1298290, 'Encoding error', 6218404, 690003, 779589, 653546, 'Encoding error', 0, 'Encoding error', 1907746, 3006520, 741113, 'Encoding error', 648040, 2423343, 3073065, 1482517, 4004560, 1386230, 3271284, 859418, 13099980, 'Encoding error', 'Encoding error', 0, 'Encoding error', 'Encoding error', 7247270, 'Encoding error', 822377, 'Encoding error', 'Encoding error', 1965278, 1941178, 'Encoding error', 2552968, 'Encoding error', 15836636, 'Encoding error', 4227672, 'Encoding error', 2176585, 2674950, 25683415, 1675

In [12]:
sum([x for x in wordcounts.values() if isinstance(x, int)])

363695216

Notes and missing API functionality?

* wanted to list only filenames (and not other HDFS file info)
* wanted to load all text files in subdirs (glob like /tmp/enron/*/*.txt)
* wanted to set encoding in .open() method
* wanted to easily read .head() of large text file
* why are the encoding errors happening?

### Example 2) Word count with hdfs3 + distributed

Start dscheduler and dworkers on nodes:

head node: `dscheduler`  
compute nodes: `dworker 172.31.56.96:8786`

In [13]:
from distributed import Executor, progress, wait
from distributed.hdfs import read_bytes

In [14]:
e = Executor('172.31.56.96:8786')

Count words in first file

In [15]:
future = e.submit(count_words, filenames[0])

In [16]:
future.result()

13099980

Count words in all (readable) files

In [17]:
futures = e.map(count_words, filenames)

In [18]:
futures[:5]

[<Future: status: finished, key: count_words-0696b727ed79bbdc95af11308fd018fc>,
 <Future: status: pending, key: count_words-621dba1d10f2901372bf24aef5a44c7e>,
 <Future: status: pending, key: count_words-8ef6b1892ae3de359533a49728bea212>,
 <Future: status: pending, key: count_words-6d30174ae9ee86144e7bead2240779a7>,
 <Future: status: pending, key: count_words-4c40f687aa80a2fc393cd38afcdfda4a>]

In [19]:
%%time
wait(futures);

CPU times: user 135 ms, sys: 13 ms, total: 148 ms
Wall time: 38.6 s


DoneAndNotDoneFutures(done={<Future: status: finished, key: count_words-8f30d393a3b0a36b83957fa8fc96686c>, <Future: status: finished, key: count_words-33d171fa945361245b189a9cc6d0990f>, <Future: status: error, key: count_words-6e21fea982141fac5f363cb0e1a0e242>, <Future: status: finished, key: count_words-09cc0fc684cf36eb82d0724666084308>, <Future: status: finished, key: count_words-22a79f763c1f44831d0d3455ef8392b9>, <Future: status: finished, key: count_words-a56400e54d74de7d73bfad7e7eeb994f>, <Future: status: finished, key: count_words-406d8bcd564ccc4a5c1a68b0e98bbfd5>, <Future: status: error, key: count_words-c396552589dbfb20e603dc8d1d542a3b>, <Future: status: error, key: count_words-103a44e7e120b8e4d28ddea9f860567c>, <Future: status: finished, key: count_words-150f1d81a105319fb2104a6b6184530f>, <Future: status: finished, key: count_words-88da8ef50f494136634b03921d510714>, <Future: status: error, key: count_words-ff8f8db94fc110d8b69ddb4857f63438>, <Future: status: error, key: count_w

In [20]:
# Some of the tasks fail due to utf-8 formatting errors, we ignore the failed tasks
futures_success = [x for x in futures if x.status == 'finished']

In [21]:
futures_success[:5]

[<Future: status: finished, key: count_words-0696b727ed79bbdc95af11308fd018fc>,
 <Future: status: finished, key: count_words-621dba1d10f2901372bf24aef5a44c7e>,
 <Future: status: finished, key: count_words-4c40f687aa80a2fc393cd38afcdfda4a>,
 <Future: status: finished, key: count_words-a3e0b012e543975bd3145c4779b79e08>,
 <Future: status: finished, key: count_words-eeede185716d6e6156fe7e69ddd9d82b>]

In [22]:
futures_success[0].result()

13099980

In [23]:
sum([x.result() for x in futures_success])

363695216

Notes and missing API functionality?

* how to view number of workers/cores?
* wanted to easily read .head() of large text file
* can I pass arguments to functions in e.map(func, input)?
* can I get the results of futures without a list comprehension, similar to an RDD?
* why are the encoding errors happnening?
* should blocking futures be default with an option to wait()?
* are these errors from dscheduler important? https://gist.github.com/koverholt/6c0f9c10b23152c3f0c4

### Example 3) Word count with hdfs3 + distributed + dask

In [24]:
import dask

Notes and missing API functionality?

* wanted to easily read .head() of large text file
* need a futures_to_bag

### Results

Time to count words in all files:

* hdfs3 - 120 seconds
* hdfs3 + distributed - 39 seconds
* hdfs3 + distributed + dask - 